In [1]:
from re import T
import re
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import os
from difflib import SequenceMatcher
import json
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

import helpers
import api

In [22]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [56]:
df_MCQ = pd.read_csv('MCQ_solved_6_sample.csv')
df_MCQ_EN = pd.read_csv('MCQ_solved_6_sample_EN.csv')
df_2013 = pd.read_csv('all_provas_enem_areas_sys_solved_sample.csv')
df_2013 = df_2013.drop(columns=['CE','DS', 'EK', 'IC', 'MR', 'TC', 'image', 'IU', 'ledor', 'figures'])
df_2013_EN = pd.read_csv('all_provas_enem_areas_sys_solved_sample_EN.csv')
df_2013_EN = df_2013_EN.drop(columns=['CE','DS', 'EK', 'IC', 'MR', 'TC', 'image', 'IU', 'ledor', 'figures'])

In [96]:
teste = pd.read_csv('all_provas_enem_areas_sys_solved.csv')

In [105]:
from datetime import datetime
date = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
csv_file = f'myfile_{date}.csv'

In [106]:
csv_file

'myfile_2024_05_16_01_40_58.csv'

In [108]:
date = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

teste.to_csv(f'results/teste_{date}.csv', index=False)

In [101]:
teste.loc[(teste['year']>=2017) & (teste['image'] == 'No')]

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image,IU,ledor,figures,description,area,cor,system_answer
440,91,2017,\nUm fato corriqueiro ao se cozinhar arr...,"\nCientificamente, sabe-se que essa mudanç...",{'A': ' reação do gás de cozinha com o ...,"{'B': ' emissão de fótons pelo sódio, excitado...",No,Yes,Yes,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS,AMARELO,NaN
443,94,2017,\nPesquisadores criaram um tipo de plaqueta ar...,\nQual a doença cujos pacientes teriam m...,"{'A': ' Filariose.\n', 'B': ' Hemofilia.\n', '...",{'B': ' Hemofilia.\n'},No,Yes,No,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS,AMARELO,NaN
446,97,2017,As centrífugas são equipamentos utilizados...,"\nNesse aparelho, a separação das substân...","{'A': ' das diferentes densidades.\n', 'B': ' ...",{'A': ' das diferentes densidades.\n'},No,Yes,Yes,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS,AMARELO,NaN
449,100,2017,\nPartículas microscópicas existentes na a...,\nA fixação de moléculas de vapor de água pelo...,"{'A': ' ligações iônicas.\n', 'B': ' interaçõe...",{'D': ' interações íon-dipolo.\n'},Yes,Yes,Yes,Yes,No,No,No,NaN,NaN,NaN,NaN,CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS,AMARELO,NaN
451,102,2017,\nA técnica do carbono-14 permite a datação de...,\nConsidere que um fragmento fóssil de massa i...,"{'A': ' 450.\n', 'B': ' 1 433.\n', 'C': ' 11 4...",{'C': ' 11 460.\n'},No,No,No,No,Yes,No,No,NaN,NaN,NaN,NaN,CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS,AMARELO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,173,2023,Uma pessoa pratica quatro atividades físicas —...,O dia da semana em que será comemorado o anive...,"{'A': 'segunda-feira.', 'B': 'terça-feira.', '...",{'C': 'quarta-feira.'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],MATEMÁTICA E SUAS TECNOLOGIAS,LARANJA,NaN
1577,174,2023,"A cada bimestre, a diretora de uma escola comp...",O desconto necessário no preço final da compra...,"{'A': '(5,0 ; 5,5).', 'B': '(8,0 ; 8,5).', 'C'...","{'A': '(5,0 ; 5,5).'}",NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],MATEMÁTICA E SUAS TECNOLOGIAS,LARANJA,NaN
1578,175,2023,Alguns estudos comprovam que os carboidratos f...,A quantidade mínima de pacotes de pão de fôrma...,"{'A': '1.', 'B': '4.', 'C': '6.', 'D': '7.', '...",{'D': '7.'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],MATEMÁTICA E SUAS TECNOLOGIAS,LARANJA,NaN
1581,178,2023,O calendário maia apresenta duas contagens sim...,"De acordo com as informações fornecidas, duran...","{'A': '741', 'B': '1 040', 'C': '1 460', 'D': ...",{'C': '1 460'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],MATEMÁTICA E SUAS TECNOLOGIAS,LARANJA,NaN


In [57]:
df_2013.columns

Index(['id', 'year', 'context', 'question', 'options', 'correct_answer',
       'description', 'area', 'cor', 'system_answer', 'steps_answer'],
      dtype='object')

#### Sample 10 questions 

In [71]:
# Padronizar os valores dos níveis de dificuldade para inglês
df_MCQ['difficult_level'] = df_MCQ['difficult_level'].map({
    'Iniciante': 'Beginner',
    'Intermediário': 'Intermediate',
    'Avançado': 'Advanced'
})

# Padronizar os valores dos níveis de Bloom para inglês
df_MCQ['bloom_level'] = df_MCQ['bloom_level'].map({
    'Analisar': 'Analyze',
    'Análise': 'Analyze',
    'Aplicar': 'Apply',
    'Aplicação': 'Apply',
    'Avaliar': 'Evaluate',
    'Avaliação': 'Evaluate',
    'Compreender': 'Understand',
    'Compreensão': 'Understand',
    'Lembrar': 'Remember',
    'Recordar': 'Remember'
})

# Padronizar os valores dos níveis de dificuldade para inglês
df_MCQ_EN['difficult_level'] = df_MCQ['difficult_level'].map({
    'Iniciante': 'Beginner',
    'Intermediário': 'Intermediate',
    'Avançado': 'Advanced'
})

# Padronizar os valores dos níveis de Bloom para inglês
df_MCQ_EN['bloom_level'] = df_MCQ['bloom_level'].map({
    'Analisar': 'Analyze',
    'Análise': 'Analyze',
    'Aplicar': 'Apply',
    'Aplicação': 'Apply',
    'Avaliar': 'Evaluate',
    'Avaliação': 'Evaluate',
    'Compreender': 'Understand',
    'Compreensão': 'Understand',
    'Lembrar': 'Remember',
    'Recordar': 'Remember'
})

df_MCQ['correct_answer'] = df_MCQ['correct_answer'].apply(lambda x: x.replace("'", ""))
df_MCQ_EN['correct_answer'] = df_MCQ_EN['correct_answer'].apply(lambda x: x.replace("'", ""))
df_2013['correct_answer'] = df_2013['correct_answer'].apply(lambda x: x.replace("'", ""))
df_2013_EN['correct_answer'] = df_2013_EN['correct_answer'].apply(lambda x: x.replace("'", ""))

In [86]:
def count_equal_and_not_equal(df):
    equal_count = (df['correct_answer'] == df['system_answer']).sum()
    not_equal_count = len(df) - equal_count
    return equal_count, not_equal_count

# DataFrames
data = {
    'df_MCQ': df_MCQ,
    'df_MCQ_EN': df_MCQ_EN,
    'df_2013': df_2013,
    'df_2013_EN': df_2013_EN
}

# Loop for para aplicar a função a cada DataFrame
for df_name, df in data.items():
    equal_count, not_equal_count = count_equal_and_not_equal(df)
    print(f"Para {df_name}:")
    print("EQUALS:", equal_count)
    print("DISTINCT:", not_equal_count)
    print()


Para df_MCQ:
EQUALS: 27
DISTINCT: 6

Para df_MCQ_EN:
EQUALS: 26
DISTINCT: 7

Para df_2013:
EQUALS: 0
DISTINCT: 10

Para df_2013_EN:
EQUALS: 3
DISTINCT: 7



In [72]:
equal_count = (df_MCQ['correct_answer'] == df_MCQ['system_answer']).sum()
print(equal_count)
not_equal_count = len(df_MCQ) - equal_count
print(not_equal_count)

27
6


In [73]:
equal_count = (df_MCQ_EN['correct_answer'] == df_MCQ_EN['system_answer']).sum()
print(equal_count)
not_equal_count = len(df_MCQ_EN) - equal_count
print(not_equal_count)

26
7


In [79]:
equal_count = (df_2013['correct_answer'] == df_2013['system_answer']).sum()
print(equal_count)
not_equal_count = len(df_2013) - equal_count
print(not_equal_count)

0
10


In [84]:
equal_count = (df_2013_EN['correct_answer'] == df_2013_EN['system_answer']).sum()
print(equal_count)
not_equal_count = len(df_2013) - equal_count
print(not_equal_count)

3
7


In [89]:
def count_matches(correct_answers, system_answers, threshold=0.95):
    match_count = 0
    not_match_count = 0

    for i in range(len(correct_answers)):
        correct_answer = correct_answers[i]
        system_answer = system_answers[i]
        ratio = similar(correct_answer, system_answer)
        
        if ratio >= threshold:
            match_count += 1
        else:
            not_match_count += 1

    return match_count, not_match_count


data = {
    'df_MCQ': df_MCQ,
    'df_MCQ_EN': df_MCQ_EN,
    'df_2013': df_2013,
    'df_2013_EN': df_2013_EN
}


for df_name, df in data.items():
    equal_count, not_equal_count = count_matches(df['correct_answer'], df['system_answer'])
    print(f"DF: {df_name}:")
    print("EQUALS:", equal_count)
    print("DISTINCT:", not_equal_count)
    print()

DF: df_MCQ:
EQUALS: 27
DISTINCT: 6

DF: df_MCQ_EN:
EQUALS: 26
DISTINCT: 7

DF: df_2013:
EQUALS: 3
DISTINCT: 7

DF: df_2013_EN:
EQUALS: 7
DISTINCT: 3



In [65]:
bloom_level_summary = df_MCQ['bloom_level'].value_counts().sort_index()
print("Bloom Level Summary: ", bloom_level_summary)
difficult_level_summary = df_MCQ['difficult_level'].value_counts().sort_index()
print("Difficult Level Summary: ", difficult_level_summary)
relevance_summary = df_MCQ['relevance'].value_counts().sort_index()
print("Relevance Summary: ", relevance_summary)
adherence_summary = df_MCQ['adherence'].value_counts().sort_index()
print("Adherence Summary: ", adherence_summary)
answerability_summary = df_MCQ['answerability'].value_counts().sort_index()
print("Answerability Summary: ", answerability_summary)
correctness_summary = df_MCQ['correctness'].value_counts().sort_index()
print("Correctness Summary: ", correctness_summary)

Bloom Level Summary:  bloom_level
Analyze       5
Apply         6
Evaluate      1
Remember      9
Understand    9
Name: count, dtype: int64
Difficult Level Summary:  difficult_level
Advanced        10
Beginner        10
Intermediate    10
Name: count, dtype: int64
Relevance Summary:  relevance
4     1
5    32
Name: count, dtype: int64
Adherence Summary:  adherence
3     1
4    24
5     8
Name: count, dtype: int64
Answerability Summary:  answerability
5    33
Name: count, dtype: int64
Correctness Summary:  correctness
1    33
Name: count, dtype: int64


In [66]:
bloom_level_summary = df_MCQ_EN['bloom_level'].value_counts().sort_index()
print("Bloom Level Summary: ", bloom_level_summary)
difficult_level_summary = df_MCQ_EN['difficult_level'].value_counts().sort_index()
print("Difficult Level Summary: ", difficult_level_summary)
relevance_summary = df_MCQ_EN['relevance'].value_counts().sort_index()
print("Relevance Summary: ", relevance_summary)
adherence_summary = df_MCQ_EN['adherence'].value_counts().sort_index()
print("Adherence Summary: ", adherence_summary)
answerability_summary = df_MCQ_EN['answerability'].value_counts().sort_index()
print("Answerability Summary: ", answerability_summary)
correctness_summary = df_MCQ_EN['correctness'].value_counts().sort_index()
print("Correctness Summary: ", correctness_summary)

Bloom Level Summary:  Series([], Name: count, dtype: int64)
Difficult Level Summary:  Series([], Name: count, dtype: int64)
Relevance Summary:  relevance
5    33
Name: count, dtype: int64
Adherence Summary:  adherence
4    33
Name: count, dtype: int64
Answerability Summary:  answerability
5    33
Name: count, dtype: int64
Correctness Summary:  correctness
0    15
1    18
Name: count, dtype: int64


In [67]:
merged_df = pd.merge(df_2013, df_2013_EN, on='id', suffixes=('_2013', '_2013_en'))

common_system_answers = merged_df[merged_df['system_answer_2013'].str[1] == merged_df['system_answer_2013_en'].str[1]]
unique_system_answers = merged_df[merged_df['system_answer_2013'].str[1] != merged_df['system_answer_2013_en'].str[1]]

num_common_ids = len(common_system_answers)
num_unique_ids = len(unique_system_answers)

print("MATCH:", num_common_ids)
print("NOT MATCH:", num_unique_ids)

MATCH: 7
NOT MATCH: 3


In [68]:
merged_df[merged_df['system_answer_2013'].str[1] != merged_df['system_answer_2013_en'].str[1]]

,id,year_2013,context_2013,question_2013,options_2013,correct_answer_2013,description_2013,area_2013,cor_2013,system_answer_2013,...,year_2013_en,context_2013_en,question_2013_en,options_2013_en,correct_answer_2013_en,description_2013_en,area_2013_en,cor_2013_en,system_answer_2013_en,steps_answer_2013_en
3,5,2013,"\nDe ponta a ponta, é tudo praia-palma, ...",\nA carta de Pero Vaz de Caminha permit...,{'A': ' Valorizar a catequese a ser real...,{'A': ' Valorizar a catequese a ser real...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{A: Valorizar a catequese a ser realizada sobr...,...,2013,"\nFrom end to end, it's all palm beach, very f...",\nThe letter from Pero Vaz de Caminha allows u...,{'A': ' To value the catechesis to be carried...,{'A': ' To value the catechesis to be carried...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{C: Transmit the knowledge of the indigenous p...,The excerpt from Pero Vaz de Caminha's letter ...
4,6,2013,\nO canto triste dos conquistados: \nos último...,"\nO texto é um registro asteca, cujo se...",{'A': ' tragédia causada pela destruição ...,{'B': ' tentativa frustrada de resistência ...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{B: tentativa frustrada de resistência a um po...,...,2013,\nThe sad song of the conquered: \nthe last da...,"\nThe text is an Aztec record, whose meaning i...",{'A': 'the tragedy caused by the destruction o...,{'B': 'the failed attempt to resist a power \n...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{A: the tragedy caused by the destruction of t...,The text describes the devastation and desolat...
9,12,2013,"\nNo final do século XIX, as Grandes So...",\nManifestações culturais como o carnaval ...,{'A': ' distinções sociais eram deixadas de l...,{'D': ' tradições populares se transformavam ...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{A: distinções sociais eram deixadas de lado e...,...,2013,"\nAt the end of the 19th century, the Great Ca...",\nCultural manifestations such as carnival als...,{'A': 'social distinctions were set aside in t...,{'D': 'popular traditions became a matter of s...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{D: popular traditions became a matter of soci...,The text mentions that the Great Carnival Soci...
